In [15]:
import torch
import pickle
import argparse
from collections import defaultdict
from util import get_image_size
from tqdm import tqdm
from unet import UNet
from model import DiffusionModel
from util import get_transforms


In [16]:
def infer(unet, diffusion_model, device, T, reverse_transform, n):


    _t = None
    i = None
    

    bottleneck = defaultdict(list)

    unet.eval()
    unet.bottleneck.register_forward_hook(lambda x, y, output: bottleneck[_t, i].append(output.cpu().numpy()))

    samples = defaultdict(list)

    with torch.no_grad():
        image = torch.randn((n, 1, 28, 28)).to(device)

        for _t in range(T):
            torch.manual_seed(_t)
            for i in reversed(range(diffusion_model.timesteps)):
                if i >= diffusion_model.timesteps - 0.1 * diffusion_model.timesteps:
                    for dropout in unet.dropouts:
                        dropout.train() 
                else:
                    for dropout in unet.dropouts:
                        dropout.eval() 
                image = diffusion_model.backward(image, torch.full((1, ), i, dtype=torch.long, device=device), unet) + 0.001
                samples[i].append(image.cpu().numpy())
    
    with open('pickels/results.pkl', 'wb') as f:
        pickle.dump(samples, f)

    with open('pickels/bottlenecks.pkl', 'wb') as f:
        pickle.dump(bottleneck, f)

In [17]:
device = 'cuda'
TIMESTEPS = 300
IMAGE_SIZE = (28, 28)
N = 2
T = 2

_, reverse_transform = get_transforms(image_size=IMAGE_SIZE)

unet = UNet(input_channels=1, output_channels=1).to(device)
unet.load_state_dict(torch.load('weight/parameters.pkl'))
diffusion_model = DiffusionModel(timesteps=TIMESTEPS)

infer(unet, diffusion_model, device, T, reverse_transform, N)

AttributeError: 'UNet' object has no attribute 'dropouts'